In [2]:
import pandas as pd

df = pd.read_sql_query('''SELECT d.document_id, d.text FROM document_text d JOIN is_syllabus i 
                                   ON d.document_id = i.document_id
                                   WHERE i.score > 0.8''',
                      'postgres:///osp')

In [3]:
df.rename(columns={'d.document_id': 'document_id',
                  'd.text': 'text'},
         inplace=True)

In [4]:
print("The corpus has {} syllabi".format(df.shape[0]))

The corpus has 557841 syllabi


In [ ]:
import textacy
corpus = textacy.corpus.Corpus('en', texts=list(df.text))

In [ ]:
terms_list = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in corpus)
doc_term_matrix, id2term = textacy.vsm.doc_term_matrix(
        terms_list, weighting='tfidf', normalize=True, smooth_idf=True,
        min_df=3, max_df=0.95, max_n_terms=200000)


In [ ]:
model = textacy.tm.TopicModel('nmf', n_topics=30)

model.fit(doc_term_matrix)